In [1]:
!pip install pyngrok
!pip install flask-cors
!ngrok config add-authtoken 2ybBrbOsYjkmgYXWQ3594lQRqT2_3PcHCPRSNvEq3cu2nM6SP

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [2]:
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-n

In [3]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from PIL import Image
import torch
import open_clip
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import base64
from io import BytesIO

# Setup
app = Flask(__name__)
CORS(app)

UPLOAD_FOLDER = "uploads"
DATASET_DIR = "/kaggle/input/similarity/d/d"  # Your dataset directory
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Load CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B/32', pretrained='openai')
model.to(device).eval()

def extract_clip_features(image_path):
    image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model.encode_image(image).cpu().numpy().flatten()
    return features

def get_image_base64(image_path):
    """Convert image to base64 string"""
    try:
        with Image.open(image_path) as img:
            # Convert to RGB if necessary
            if img.mode != 'RGB':
                img = img.convert('RGB')
            # Resize if image is too large
            max_size = (800, 800)  # Maximum dimensions
            if img.size[0] > max_size[0] or img.size[1] > max_size[1]:
                img.thumbnail(max_size, Image.Resampling.LANCZOS)
            # Save to buffer
            buffer = BytesIO()
            img.save(buffer, format='JPEG', quality=85)  # Use JPEG for smaller size
            img_str = base64.b64encode(buffer.getvalue()).decode()
            return f"data:image/jpeg;base64,{img_str}"
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

@app.route('/match', methods=['POST'])
def match_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image uploaded'}), 400

    image_file = request.files['image']
    if image_file.filename == '':
        return jsonify({'error': 'Empty filename'}), 400

    filename = secure_filename(image_file.filename)
    uploaded_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    image_file.save(uploaded_path)

    try:
        # Extract features from uploaded image
        generated_features = extract_clip_features(uploaded_path)

        image_features = {}
        image_paths = []

        # Process all images in the dataset
        for root, _, files in os.walk(DATASET_DIR):
            for fname in files:
                if fname.lower().endswith(tuple(ALLOWED_EXTENSIONS)):
                    img_path = os.path.join(root, fname)
                    try:
                        features = extract_clip_features(img_path)
                        rel_path = os.path.relpath(img_path, DATASET_DIR)
                        image_features[rel_path] = features
                        image_paths.append(img_path)  # Store full path for later
                    except Exception as e:
                        print(f"Error processing {img_path}: {e}")

        features_matrix = np.array(list(image_features.values()))
        similarities = cosine_similarity([generated_features], features_matrix)[0]

        threshold = 0.75
        indices_above = [i for i, sim in enumerate(similarities) if sim >= threshold]

        if indices_above:
            sorted_indices = sorted(indices_above, key=lambda i: similarities[i], reverse=True)
            top_indices = sorted_indices[:3]
            
            results = []
            for i in top_indices:
                img_path = image_paths[i]
                img_base64 = get_image_base64(img_path)
                if img_base64:
                    results.append({
                        "filename": os.path.basename(img_path),
                        "similarity": float(similarities[i]),
                        "url": img_base64  # Send image as base64
                    })
            
            return jsonify(results=results)
        else:
            return jsonify(results=[])

    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        # Clean up uploaded file
        if os.path.exists(uploaded_path):
            os.remove(uploaded_path)

# Run with ngrok
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(" * ngrok tunnel: ", public_url)
    app.run()

open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


 * ngrok tunnel:  NgrokTunnel: "https://7d5a-34-58-83-46.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
